In [226]:
import glob
import cv2

import matplotlib.pyplot as plt
%matplotlib inline
from pandas.core.common import flatten
import numpy as np
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

import learn2learn as l2l
from losses import *
from Triplets import *
from cnn4_triplet import *

import datasets
import networks

In [227]:
Triplet_Model_Parameter = {
    "CIFARFS" : {"data" : TripletFSCIFAR100 , "root" : "~/data", "download": True , "transform" : transforms.Compose([transforms.ToTensor()]), "hidden_size":64, "layers":4, "channels":3, "max_pool":True, "embedding_size":256,"margin":1.0,"lambda":1} ,
    "CUB" : {"data" : TripletCUB , "root" : "./data", "download": True , "transform" : transforms.Compose([transforms.ToTensor()]), "hidden_size":64, "layers":4, "channels":3, "max_pool":True, "embedding_size":1600,"margin":1.0,"lambda":1},
    "FLOWERS" : {"data" : TripletFlowers , "root" : "~/data", "download": True , "transform" : transforms.Compose([transforms.ToTensor()]), "hidden_size":64, "layers":4, "channels":3, "max_pool":True, "embedding_size":1600,"margin":1.0,"lambda":1},
    "MINIIMAGENET" : {"data" : TripletMiniImageNet , "root" : "data", "download": True , "transform" : transforms.Compose([transforms.ToTensor()]), "hidden_size":32, "layers":4, "channels":3, "max_pool":True, "embedding_size":800,"margin":1.0,"lambda":1},
    "OMNIGLOT" : {"data" : TripletOmniglot , "root" : "~/data", "download": True , "transform" : transforms.Compose([transforms.ToTensor(),transforms.Resize((28,28))]), "hidden_size":64, "layers":4, "channels":1, "max_pool":False, "embedding_size":256,"margin":1.0,"lambda":1},
    "MINIIMAGENET_64" : {"data" : TripletMiniImageNet , "root" : "data", "download": True , "transform" : transforms.Compose([transforms.ToTensor()]), "hidden_size":64, "layers":4, "channels":3, "max_pool":True, "embedding_size":1600,"margin":1.0,"lambda":1},

}

In [228]:
def accuracy(predictions, targets, shots):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    # return (predictions == targets).sum().float() / targets.size(0)

    if shots == 1:
        mask = np.array([True, False, False, False, False, False, False, False, True, True, True, True])
    else: # shots==5
        mask = np.array(
        [True, False, False, False, False, False, False, False, False, False, False, False,	False, False, False, False, False, False, False, False, 
        True, False, False, False, False, True, False, False, False, False, True, False, False, False, False, True, False, False, False, False,
        True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True ,True, True])
    return (predictions[mask] == targets[mask]).sum().float() / targets[mask].size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    adaptation_data, adaptation_labels, evaluation_data, evaluation_labels = batch
    adaptation_data[0]= adaptation_data[0].to(device)
    adaptation_data[1]= adaptation_data[1].to(device)
    adaptation_data[2]= adaptation_data[2].to(device)

    adaptation_labels= adaptation_labels.to(device)

    evaluation_data[0]= evaluation_data[0].to(device)
    evaluation_data[1]= evaluation_data[1].to(device)
    evaluation_data[2]= evaluation_data[2].to(device)

    evaluation_labels= evaluation_labels.to(device)

    # Adapt the model
    for step in range(adaptation_steps):
        out= learner(adaptation_data[0], adaptation_data[1], adaptation_data[2] )
        train_error = loss(out[0],out[1],out[2], torch.vstack([out[3],out[4],out[5]]), torch.hstack([adaptation_labels[0],adaptation_labels[1],adaptation_labels[2]]))
        learner.adapt(train_error, allow_unused=True)

    # Evaluate the adapted model
    predictions = learner(evaluation_data[0], evaluation_data[1], evaluation_data[2])
    valid_error= loss(predictions[0],predictions[1],predictions[2], torch.vstack([predictions[3],predictions[4],predictions[5]]), torch.hstack([evaluation_labels[0],evaluation_labels[1],evaluation_labels[2]]))
    valid_accuracy = accuracy(torch.vstack([predictions[3],predictions[4],predictions[5]]), torch.hstack([evaluation_labels[0],evaluation_labels[1],evaluation_labels[2]]), shots)
    return valid_error, valid_accuracy

def fast_adapt_image_retrieval(batch, learner, loss, adaptation_steps, shots, ways, device):
    adaptation_data, adaptation_labels, evaluation_data, evaluation_labels = batch
    adaptation_data[0]= adaptation_data[0].to(device)
    adaptation_data[1]= adaptation_data[1].to(device)
    adaptation_data[2]= adaptation_data[2].to(device)

    adaptation_labels= adaptation_labels.to(device)

    

    # Adapt the model
    for step in range(adaptation_steps):
        out= learner(adaptation_data[0], adaptation_data[1], adaptation_data[2] )
        train_error = loss(out[0],out[1],out[2], torch.vstack([out[3],out[4],out[5]]), torch.hstack([adaptation_labels[0],adaptation_labels[1],adaptation_labels[2]]))
        learner.adapt(train_error, allow_unused=True)

   
    return train_error, train_error

In [229]:
ways=5
shots=1
meta_lr=0.001 # as in MAML
fast_lr=0.01 # as in MAML
meta_batch_size=4 # Maml Omniglot:32; miniImageNet: 4 
adaptation_steps=15
test_adaptation_steps=10
num_iterations= 60000
cuda=True
seed=42
num_test_episodes= 600
selected_model = "MINIIMAGENET_64"

In [230]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device('cpu')

if cuda:
    torch.cuda.manual_seed(seed)
    device = torch.device('cuda')    

triplet_imagenet_dataset = Triplet_Model_Parameter[selected_model]["data"](root = Triplet_Model_Parameter[selected_model]["root"], download = Triplet_Model_Parameter[selected_model]["download"], transform = Triplet_Model_Parameter[selected_model]["transform"])

# Create model using saved parameters:
model = TripletCNN4(output_size= ways, hidden_size=Triplet_Model_Parameter[selected_model]["hidden_size"], layers=Triplet_Model_Parameter[selected_model]["layers"], channels=Triplet_Model_Parameter[selected_model]["channels"], max_pool=Triplet_Model_Parameter[selected_model]["max_pool"], embedding_size=Triplet_Model_Parameter[selected_model]["embedding_size"])

try : 
    model = torch.load("./Tripletmaml_MINIIMAGENET_RetrievalTest_batchsize4_shots1.pt")
    model.to(device)
except : 
    model = TripletCNN4(output_size= ways, hidden_size=Triplet_Model_Parameter[selected_model]["hidden_size"], layers=Triplet_Model_Parameter[selected_model]["layers"], channels=Triplet_Model_Parameter[selected_model]["channels"], max_pool=Triplet_Model_Parameter[selected_model]["max_pool"], embedding_size=Triplet_Model_Parameter[selected_model]["embedding_size"])
    model.load_state_dict(torch.load("./Tripletmaml_MINIIMAGENET_RetrievalTest_batchsize4_shots1.pt"))
    model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=True)

In [231]:
opt = optim.Adam(maml.parameters(), meta_lr) # meta-update
triplet_w= Triplet_Model_Parameter[selected_model]["lambda"]
combined_loss_fn= CombinedLoss2(triplet_w, shots)

In [232]:
a = triplet_imagenet_dataset.sample("test",mode="img_retrieval",samples_per_class = 10)

In [233]:
x_support = a[0]
y_support = a[1]
x_query   = a[2]
y_query   = a[3]

In [234]:
x_support[0].shape

torch.Size([4, 3, 84, 84])

In [235]:
x_support[0][0].shape

torch.Size([3, 84, 84])

In [236]:
x_support[0][0][0].shape

torch.Size([84, 84])

In [237]:
opt.zero_grad() # for each batch, gradients should be cleaned.
# Compute meta-training loss
# print('Task no: ', task)
learner = maml.clone()
batch = a

train_error, _ = fast_adapt_image_retrieval(batch,
                                                    learner,
                                                    combined_loss_fn,
                                                    adaptation_steps,
                                                    shots,
                                                    ways,
                                                    device)                                                
embeddings = []
labels = torch.from_numpy(np.array(list(a[3])))

In [238]:
for idx,image in enumerate(a[2]):
    embedding,class_prob =learner.forward_once(torch.unsqueeze(image,0).to(device)) 
    embedding = F.normalize(embedding, p = 2 , dim =1).cpu()
    embeddings.append(np.array(embedding[0].tolist()))


In [239]:
embeddings = torch.from_numpy(np.array(embeddings))

In [240]:
embeddings.shape

torch.Size([50, 1600])

In [241]:
labels.shape

torch.Size([50])

In [242]:
def recall(Fvec, imgLab,rank=None):
    # imgLab numpy.ndarray of shape: (8041,)
   
    N = len(imgLab) #8041 labels

    imgLab = torch.LongTensor([imgLab[i] for i in range(len(imgLab))])
    # imgLab.shape: [8041]
    # Fvec.shape: [8041, 128]
    
    D = Fvec.mm(torch.t(Fvec)) # mm: matrix multiplication. (n×m) mm (m×p) results in  (n×p) tensor.
    # [8041, 128] mm [128, 8041] --> [8041, 8041] this is D matrix
    # There are 1's along the diagonal!
    
    D[torch.eye(len(imgLab)).bool()] = -1 
    # torch.eye: Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    # D[torch.eye(len(imgLab)).bool()]: diagonal elements of D will take a value of -1 ; the rest will remain the same
    
    if rank==None: # only rank 1 is computed
        _,idx = D.max(1) # returns both values and indices; dim=1 means returns for each row 
        imgPre = imgLab[idx]
        A = (imgPre==imgLab).float()
        return (torch.sum(A)/N).item()
    else:
        _,idx = D.topk(rank[-1])
        acc_list = []
        for r in rank:
            A = 0
            for i in range(r):
                imgPre = imgLab[idx[:,i]]
                A += (imgPre==imgLab).float()
            acc_list.append((torch.sum((A>0).float())/N).item())
        return torch.Tensor(acc_list)

In [243]:
recall(embeddings, labels, rank= [1, 2, 4, 8,9, 10]) 

tensor([0.2800, 0.4200, 0.6000, 0.8600, 0.8800, 0.8800])

In [244]:
recall_values = []

for i in range(600):
    print(i)
    a = triplet_imagenet_dataset.sample("test",mode="img_retrieval",samples_per_class = 10)
    opt.zero_grad() # for each batch, gradients should be cleaned.
    # Compute meta-training loss
    # print('Task no: ', task)
    learner = maml.clone()
    batch = a

    train_error, _ = fast_adapt_image_retrieval(batch,
                                                        learner,
                                                        combined_loss_fn,
                                                        adaptation_steps,
                                                        shots,
                                                        ways,
                                                        device)                                                
    embeddings = []
    labels = torch.from_numpy(np.array(list(a[3])))
    for idx,image in enumerate(a[2]):
        embedding,class_prob =learner.forward_once(torch.unsqueeze(image,0).to(device)) 
        embedding = F.normalize(embedding, p = 2 , dim =1).cpu()
        embeddings.append(np.array(embedding[0].tolist()))
    embeddings = torch.from_numpy(np.array(embeddings))
    recall_values.append(recall(embeddings, labels, rank= [1, 2, 4, 8, 9,  10]) )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [245]:
recall_values

[tensor([0.2800, 0.3600, 0.6200, 0.8800, 0.9200, 0.9200]),
 tensor([0.3000, 0.4000, 0.5600, 0.8000, 0.8400, 0.8800]),
 tensor([0.4200, 0.5200, 0.6800, 0.8600, 0.9200, 0.9200]),
 tensor([0.2600, 0.3800, 0.7000, 0.8600, 0.8600, 0.8800]),
 tensor([0.2800, 0.3800, 0.5400, 0.7800, 0.7800, 0.8000]),
 tensor([0.2600, 0.3400, 0.5400, 0.8000, 0.8600, 0.9000]),
 tensor([0.2400, 0.4200, 0.5600, 0.8000, 0.8600, 0.9200]),
 tensor([0.1600, 0.3200, 0.5600, 0.8800, 0.9200, 0.9400]),
 tensor([0.1200, 0.3400, 0.5400, 0.8400, 0.9000, 0.9000]),
 tensor([0.2000, 0.3800, 0.6200, 0.8200, 0.8200, 0.8200]),
 tensor([0.1400, 0.3200, 0.5200, 0.7800, 0.8000, 0.8400]),
 tensor([0.2200, 0.3200, 0.5000, 0.8200, 0.8600, 0.9400]),
 tensor([0.1800, 0.3600, 0.5600, 0.8000, 0.8000, 0.8000]),
 tensor([0.3200, 0.4000, 0.5800, 0.7800, 0.8200, 0.8200]),
 tensor([0.2800, 0.4400, 0.6400, 0.8200, 0.8800, 0.8800]),
 tensor([0.2800, 0.3800, 0.5600, 0.8600, 0.8600, 0.8800]),
 tensor([0.2000, 0.3800, 0.5200, 0.7600, 0.7600, 0.7800]

In [246]:
mean_recall_at_k = [[],[],[],[],[],[]]
for a in recall_values:
    mean_recall_at_k[0].append(a[0])
    mean_recall_at_k[1].append(a[1])
    mean_recall_at_k[2].append(a[2])
    mean_recall_at_k[3].append(a[3])
    mean_recall_at_k[4].append(a[4])
    mean_recall_at_k[5].append(a[5])

In [247]:
for i in mean_recall_at_k:
    print(np.mean(i))

0.27323332
0.41893333
0.60836667
0.8074
0.83940005
0.8641334


In [248]:
def precision_at_k(y_true, y_pred, k=12):
    """ Computes Precision at k for one sample
    
    Parameters
    __________
    y_true: np.array
            Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations
            
    Returns
    _______
    score: double
           Precision at k
    """
    intersection = np.intersect1d(y_true, y_pred[:k])
    return len(intersection) / k

def rel_at_k(y_true, y_pred, k=12):
    """ Computes Relevance at k for one sample
    
    Parameters
    __________
    y_true: np.array
            Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations
            
    Returns
    _______
    score: double
           Relevance at k
    """
    if y_pred[k-1] in y_true:
        return 1
    else:
        return 0

def average_precision_at_k(y_true, y_pred, k=12):
    """ Computes Average Precision at k for one sample
    
    Parameters
    __________
    y_true: np.array
            Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations
            
    Returns
    _______
    score: double
           Average Precision at k
    """
    ap = 0.0
    rel_counter = 0
    for i in range(1, k+1):
        ap += precision_at_k(y_true, y_pred, i) * rel_at_k(y_true, y_pred, i)
        rel_counter += rel_at_k(y_true, y_pred, i)
    #return ap / min(k, len(y_true))
    return ap / rel_counter


def mean_average_precision(y_true, y_pred, k=12):
    """ Computes MAP at k
    
    Parameters
    __________
    y_true: np.array
            2D Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            2D Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations
            
    Returns
    _______
    score: double
           MAP at k
    """

    return np.mean([average_precision_at_k(gt, pred, k) \
                    for gt, pred in zip(y_true, y_pred)])

In [282]:
def mAP_at_k(Fvec, imgLab,gt,rank=1):
    # imgLab numpy.ndarray of shape: (8041,)
   
    N = len(imgLab) #8041 labels

    imgLab = torch.LongTensor([imgLab[i] for i in range(len(imgLab))])
    # imgLab.shape: [8041]
    # Fvec.shape: [8041, 128]
    
    D = Fvec.mm(torch.t(Fvec)) # mm: matrix multiplication. (n×m) mm (m×p) results in  (n×p) tensor.
    # [8041, 128] mm [128, 8041] --> [8041, 8041] this is D matrix
    # There are 1's along the diagonal!
    
    D[torch.eye(len(imgLab)).bool()] = -1 
    # torch.eye: Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    # D[torch.eye(len(imgLab)).bool()]: diagonal elements of D will take a value of -1 ; the rest will remain the same
    
    #print("Distance Matris Shape" , D.shape)
    print(D)
    _,idx = D.topk(rank[-1])


    return mean_average_precision(np.array(gt),idx.numpy(), k= rank[-1])

In [250]:
import math
y_true = []
for i in  range(50):
    floor = math.floor(i/10) 
    max = ((floor + 1 ) * 10 ) 

    print("for i.th element" , str(i) , " floor is " , floor , "max is ", max , "floor * 10 = " , floor*10 )
    range_list = list(range(floor*10 , max))
    range_list.remove(i)
    y_true.append(range_list)
    

for i.th element 0  floor is  0 max is  10 floor * 10 =  0
for i.th element 1  floor is  0 max is  10 floor * 10 =  0
for i.th element 2  floor is  0 max is  10 floor * 10 =  0
for i.th element 3  floor is  0 max is  10 floor * 10 =  0
for i.th element 4  floor is  0 max is  10 floor * 10 =  0
for i.th element 5  floor is  0 max is  10 floor * 10 =  0
for i.th element 6  floor is  0 max is  10 floor * 10 =  0
for i.th element 7  floor is  0 max is  10 floor * 10 =  0
for i.th element 8  floor is  0 max is  10 floor * 10 =  0
for i.th element 9  floor is  0 max is  10 floor * 10 =  0
for i.th element 10  floor is  1 max is  20 floor * 10 =  10
for i.th element 11  floor is  1 max is  20 floor * 10 =  10
for i.th element 12  floor is  1 max is  20 floor * 10 =  10
for i.th element 13  floor is  1 max is  20 floor * 10 =  10
for i.th element 14  floor is  1 max is  20 floor * 10 =  10
for i.th element 15  floor is  1 max is  20 floor * 10 =  10
for i.th element 16  floor is  1 max is  20 

In [251]:
y_true

[[1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8],
 [11, 12, 13, 14, 15, 16, 17, 18, 19],
 [10, 12, 13, 14, 15, 16, 17, 18, 19],
 [10, 11, 13, 14, 15, 16, 17, 18, 19],
 [10, 11, 12, 14, 15, 16, 17, 18, 19],
 [10, 11, 12, 13, 15, 16, 17, 18, 19],
 [10, 11, 12, 13, 14, 16, 17, 18, 19],
 [10, 11, 12, 13, 14, 15, 17, 18, 19],
 [10, 11, 12, 13, 14, 15, 16, 18, 19],
 [10, 11, 12, 13, 14, 15, 16, 17, 19],
 [10, 11, 12, 13, 14, 15, 16, 17, 18],
 [21, 22, 23, 24, 25, 26, 27, 28, 29],
 [20, 22, 23, 24, 25, 26, 27, 28, 29],
 [20, 21, 23, 24, 25, 26, 27, 28, 29],
 [20, 21, 22, 24, 25, 26, 27, 28, 29],
 [20, 21, 22, 23, 25, 26, 27, 28, 29],
 [20, 21, 22, 23, 24, 26, 27, 28, 29],
 [20, 21, 22, 23, 24, 25, 27, 28, 29],
 [20, 21, 22, 23, 24, 25, 26, 28, 29]

In [252]:
mean_map = []
for i in range(600):
    print(i)
    a = triplet_imagenet_dataset.sample("test",mode="img_retrieval",samples_per_class = 10)
    opt.zero_grad() # for each batch, gradients should be cleaned.
    # Compute meta-training loss
    # print('Task no: ', task)
    learner = maml.clone()
    batch = a

    train_error, _ = fast_adapt_image_retrieval(batch,
                                                        learner,
                                                        combined_loss_fn,
                                                        adaptation_steps,
                                                        shots,
                                                        ways,
                                                        device)                                                
    embeddings = []
    labels = torch.from_numpy(np.array(list(a[3])))
    for idx,image in enumerate(a[2]):
        embedding,class_prob =learner.forward_once(torch.unsqueeze(image,0).to(device)) 
        embedding = F.normalize(embedding, p = 2 , dim =1).cpu()
        embeddings.append(np.array(embedding[0].tolist()))
    embeddings = torch.from_numpy(np.array(embeddings))

    mean_map.append(mAP_at_k(embeddings,labels,y_true,[9]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [253]:
np.mean(mean_map)

0.11892957818930043

In [268]:
mean_map

[0.08166754850088184,
 0.14786243386243386,
 0.13549647266313933,
 0.12651675485008818,
 0.12207936507936505,
 0.1723809523809524,
 0.07552469135802468,
 0.10566754850088184,
 0.11682892416225751,
 0.16264814814814815,
 0.13576543209876543,
 0.10209347442680775,
 0.09438007054673721,
 0.11770899470899471,
 0.10703262786596122,
 0.12149029982363314,
 0.1182689594356261,
 0.13524603174603175,
 0.11179894179894179,
 0.08317460317460316,
 0.09112698412698414,
 0.09719664902998236,
 0.11578395061728396,
 0.10047971781305114,
 0.1503439153439153,
 0.13501940035273366,
 0.12100088183421515,
 0.1255679012345679,
 0.15276719576719575,
 0.11666931216931217,
 0.16171869488536153,
 0.09003174603174602,
 0.09621252204585538,
 0.14277072310405642,
 0.12371252204585537,
 0.1158174603174603,
 0.10629100529100528,
 0.137347442680776,
 0.11038447971781302,
 0.12893562610229278,
 0.1252821869488536,
 0.11294797178130513,
 0.11006878306878307,
 0.08047354497354497,
 0.14390740740740743,
 0.126447089947089

In [254]:
def mtest(gt , target ,labels) : 
    print(gt)
    print(target)
    print(target[0].numpy())
    print(labels[target[0].numpy()])
   

In [258]:
def mAP_at_k(Fvec, imgLab,gt,rank=1):
    # imgLab numpy.ndarray of shape: (8041,)
   
    N = len(imgLab) #8041 labels

    imgLab = torch.LongTensor([imgLab[i] for i in range(len(imgLab))])
    # imgLab.shape: [8041]
    # Fvec.shape: [8041, 128]
    
    D = Fvec.mm(torch.t(Fvec)) # mm: matrix multiplication. (n×m) mm (m×p) results in  (n×p) tensor.
    # [8041, 128] mm [128, 8041] --> [8041, 8041] this is D matrix
    # There are 1's along the diagonal!
    
    D[torch.eye(len(imgLab)).bool()] = -1 
    # torch.eye: Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    # D[torch.eye(len(imgLab)).bool()]: diagonal elements of D will take a value of -1 ; the rest will remain the same
    
    #print("Distance Matris Shape" , D.shape)

    _,idx = D.topk(rank[-1])

    mtest(y_true,idx,labels)
    return mean_average_precision(np.array(gt),idx.numpy(), k= rank[-1])

In [259]:
mtest(y_true,embeddings,labels)

[[1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8], [11, 12, 13, 14, 15, 16, 17, 18, 19], [10, 12, 13, 14, 15, 16, 17, 18, 19], [10, 11, 13, 14, 15, 16, 17, 18, 19], [10, 11, 12, 14, 15, 16, 17, 18, 19], [10, 11, 12, 13, 15, 16, 17, 18, 19], [10, 11, 12, 13, 14, 16, 17, 18, 19], [10, 11, 12, 13, 14, 15, 17, 18, 19], [10, 11, 12, 13, 14, 15, 16, 18, 19], [10, 11, 12, 13, 14, 15, 16, 17, 19], [10, 11, 12, 13, 14, 15, 16, 17, 18], [21, 22, 23, 24, 25, 26, 27, 28, 29], [20, 22, 23, 24, 25, 26, 27, 28, 29], [20, 21, 23, 24, 25, 26, 27, 28, 29], [20, 21, 22, 24, 25, 26, 27, 28, 29], [20, 21, 22, 23, 25, 26, 27, 28, 29], [20, 21, 22, 23, 24, 26, 27, 28, 29], [20, 21, 22, 23, 24, 25, 27, 28, 29], [20, 21, 22, 23, 24, 25, 26, 28, 29], [20, 21, 22, 23, 24, 25, 

In [260]:
mAP_at_k(embeddings,labels,y_true,[9])

[[1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8], [11, 12, 13, 14, 15, 16, 17, 18, 19], [10, 12, 13, 14, 15, 16, 17, 18, 19], [10, 11, 13, 14, 15, 16, 17, 18, 19], [10, 11, 12, 14, 15, 16, 17, 18, 19], [10, 11, 12, 13, 15, 16, 17, 18, 19], [10, 11, 12, 13, 14, 16, 17, 18, 19], [10, 11, 12, 13, 14, 15, 17, 18, 19], [10, 11, 12, 13, 14, 15, 16, 18, 19], [10, 11, 12, 13, 14, 15, 16, 17, 19], [10, 11, 12, 13, 14, 15, 16, 17, 18], [21, 22, 23, 24, 25, 26, 27, 28, 29], [20, 22, 23, 24, 25, 26, 27, 28, 29], [20, 21, 23, 24, 25, 26, 27, 28, 29], [20, 21, 22, 24, 25, 26, 27, 28, 29], [20, 21, 22, 23, 25, 26, 27, 28, 29], [20, 21, 22, 23, 24, 26, 27, 28, 29], [20, 21, 22, 23, 24, 25, 27, 28, 29], [20, 21, 22, 23, 24, 25, 26, 28, 29], [20, 21, 22, 23, 24, 25, 

0.11929805996472664

In [261]:
GT = [1,2,3,4,5,6,7,8,9] # ilk eleman için
pred0 = [2,32,6,34,5,17,27,9,8]



1.0

In [ ]:
mAP_at_k

In [262]:
print(average_precision_at_k(GT,pred0,k=1))
print(average_precision_at_k(GT,pred0,k=2))
print(average_precision_at_k(GT,pred0,k=3))
print(average_precision_at_k(GT,pred0,k=4))
print(average_precision_at_k(GT,pred0,k=5))
print(average_precision_at_k(GT,pred0,k=6))
print(average_precision_at_k(GT,pred0,k=7))
print(average_precision_at_k(GT,pred0,k=8))
print(average_precision_at_k(GT,pred0,k=9))

1.0
0.5
0.5555555555555555
0.41666666666666663
0.4533333333333333
0.37777777777777777
0.3238095238095238
0.3458333333333333
0.3691358024691358


In [265]:
GT = [1,2,3,4,5,6,7,8,9] # ilk eleman için
pred0 = [2,32,6,34,5,17,27,9,8]
print(precision_at_k(GT,pred0,k=1) , rel_at_k(GT,pred0,k=1))
print(precision_at_k(GT,pred0,k=2), rel_at_k(GT,pred0,k=2))
print(precision_at_k(GT,pred0,k=3), rel_at_k(GT,pred0,k=3))
print(precision_at_k(GT,pred0,k=4), rel_at_k(GT,pred0,k=4))
print(precision_at_k(GT,pred0,k=5), rel_at_k(GT,pred0,k=5))
print(precision_at_k(GT,pred0,k=6), rel_at_k(GT,pred0,k=6))
print(precision_at_k(GT,pred0,k=7), rel_at_k(GT,pred0,k=7))
print(precision_at_k(GT,pred0,k=8), rel_at_k(GT,pred0,k=8))
print(precision_at_k(GT,pred0,k=9), rel_at_k(GT,pred0,k=9))

1.0 1
0.5 0
0.6666666666666666 1
0.5 0
0.6 1
0.5 0
0.42857142857142855 0
0.5 1
0.5555555555555556 1


In [266]:
1.666666 / 4 

0.4166665

In [267]:
y_true

[[1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8],
 [11, 12, 13, 14, 15, 16, 17, 18, 19],
 [10, 12, 13, 14, 15, 16, 17, 18, 19],
 [10, 11, 13, 14, 15, 16, 17, 18, 19],
 [10, 11, 12, 14, 15, 16, 17, 18, 19],
 [10, 11, 12, 13, 15, 16, 17, 18, 19],
 [10, 11, 12, 13, 14, 16, 17, 18, 19],
 [10, 11, 12, 13, 14, 15, 17, 18, 19],
 [10, 11, 12, 13, 14, 15, 16, 18, 19],
 [10, 11, 12, 13, 14, 15, 16, 17, 19],
 [10, 11, 12, 13, 14, 15, 16, 17, 18],
 [21, 22, 23, 24, 25, 26, 27, 28, 29],
 [20, 22, 23, 24, 25, 26, 27, 28, 29],
 [20, 21, 23, 24, 25, 26, 27, 28, 29],
 [20, 21, 22, 24, 25, 26, 27, 28, 29],
 [20, 21, 22, 23, 25, 26, 27, 28, 29],
 [20, 21, 22, 23, 24, 26, 27, 28, 29],
 [20, 21, 22, 23, 24, 25, 27, 28, 29],
 [20, 21, 22, 23, 24, 25, 26, 28, 29]

In [269]:
test_sample =  triplet_imagenet_dataset.sample("test",mode="img_retrieval",samples_per_class = 10)

In [271]:
x_query = test_sample[2]
y_query = test_sample[3]

In [278]:
np.array(x_query).shape

/tmp/ipykernel_318846/1144268499.py:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.array(x_query).shape
/tmp/ipykernel_318846/1144268499.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(x_query).shape


(50,)

In [281]:
from torchmetrics import RetrievalMAP

indexes = 


target = [rel_at_k() for i in ra]